In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import post_pass

In [2]:
cov_csv = "Resources/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv"

In [3]:
cases_df = pd.read_csv(cov_csv)

cases_df.head()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,3/11/2021,KS,297229,241035.0,56194.0,0,0.0,4851,NaN,NaN,0,0.0,3/12/2021 15:20,Agree,NaN
1,3/19/2020,FL,386,NaN,NaN,76,0.0,12,NaN,NaN,2,0.0,3/19/2020 0:00,Not agree,Not agree
2,6/11/2021,TX,2965966,NaN,NaN,1463,355.0,51158,NaN,NaN,17,0.0,6/13/2021 0:00,Not agree,Not agree
3,3/1/2021,CO,438745,411869.0,26876.0,677,60.0,5952,5218.0,734.0,1,0.0,3/1/2021 0:00,Agree,Agree
4,8/22/2020,AR,56199,NaN,NaN,547,0.0,674,NaN,NaN,11,0.0,8/23/2020 14:15,Not agree,Not agree


In [12]:
#convert date time column
cases_df['submission_date']=pd.to_datetime(cases_df['submission_date'])


In [15]:
#drop all dates other than the first on the month
fil_cases_df = cases_df[pd.to_datetime(cases_df["submission_date"]).dt.is_month_start]


In [16]:
fil_cases_df

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
3,2021-03-01,CO,438745,411869.0,26876.0,677,60.0,5952,5218.0,734.0,1,0.0,3/1/2021 0:00,Agree,Agree
18,2021-08-01,AS,0,NaN,NaN,0,0.0,0,NaN,NaN,0,0.0,8/2/2021 14:03,NaN,NaN
67,2021-04-01,KS,302372,244543.0,57829.0,0,173.0,4913,NaN,NaN,0,2.0,4/2/2021 13:56,Agree,NaN
108,2021-03-01,OK,433438,342765.0,90673.0,270,69.0,6925,5892.0,1033.0,12,0.0,3/2/2021 0:00,Agree,Agree
141,2020-05-01,UT,4828,4828.0,0.0,75,0.0,46,46.0,0.0,0,0.0,5/2/2020 15:47,Agree,Agree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38448,2021-04-01,NM,191945,NaN,NaN,293,0.0,4188,NaN,NaN,6,0.0,4/3/2021 0:00,NaN,Not agree
38458,2021-05-01,NY,1121015,NaN,NaN,1857,0.0,19409,NaN,NaN,0,0.0,5/2/2021 14:13,Not agree,Not agree
38462,2021-10-01,SD,145474,NaN,NaN,456,99.0,2145,1767.0,378.0,3,0.0,10/2/2021 13:37,NaN,Agree
38476,2021-07-01,TN,867223,713964.0,153259.0,253,73.0,12612,10158.0,2454.0,4,1.0,6/30/2021 0:00,Agree,Agree


In [17]:
#order by submis date 
fil_cases_df = fil_cases_df.sort_values(by=['state', 'submission_date'])

In [18]:
fil_cases_df = fil_cases_df[["submission_date", "state", "tot_cases", "new_case", "tot_death", "new_death"]]

In [19]:
fil_cases_df

,submission_date,state,tot_cases,new_case,tot_death,new_death
923,2020-02-01,AK,0,0,0,0
872,2020-03-01,AK,0,0,0,0
277,2020-04-01,AK,141,13,6,0
34637,2020-05-01,AK,364,9,10,1
2242,2020-06-01,AK,470,7,13,0
...,...,...,...,...,...,...
17761,2021-06-01,WY,60364,58,720,1
18930,2021-07-01,WY,62353,63,747,0
29886,2021-08-01,WY,65351,112,776,0
17040,2021-09-01,WY,76007,871,858,0


In [20]:
fil_cases_df = fil_cases_df.loc[fil_cases_df['state'].isin(["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NYC", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"])]

In [21]:
fil_cases_df = fil_cases_df.replace("NYC", "NY")
fil_cases_df

,submission_date,state,tot_cases,new_case,tot_death,new_death
923,2020-02-01,AK,0,0,0,0
872,2020-03-01,AK,0,0,0,0
277,2020-04-01,AK,141,13,6,0
34637,2020-05-01,AK,364,9,10,1
2242,2020-06-01,AK,470,7,13,0
...,...,...,...,...,...,...
17761,2021-06-01,WY,60364,58,720,1
18930,2021-07-01,WY,62353,63,747,0
29886,2021-08-01,WY,65351,112,776,0
17040,2021-09-01,WY,76007,871,858,0


In [22]:
final_df = fil_cases_df
final_df

,submission_date,state,tot_cases,new_case,tot_death,new_death
923,2020-02-01,AK,0,0,0,0
872,2020-03-01,AK,0,0,0,0
277,2020-04-01,AK,141,13,6,0
34637,2020-05-01,AK,364,9,10,1
2242,2020-06-01,AK,470,7,13,0
...,...,...,...,...,...,...
17761,2021-06-01,WY,60364,58,720,1
18930,2021-07-01,WY,62353,63,747,0
29886,2021-08-01,WY,65351,112,776,0
17040,2021-09-01,WY,76007,871,858,0


In [23]:
connection_string = f"postgres:{post_pass}@localhost:5432/Project2"
engine = create_engine(f'postgresql://{connection_string}') 

In [24]:
engine.table_names()

['us_unemployment', 'state_population', 'covid_case_death', 'covid_vac']

In [25]:
#add data to sql table
#final_df.to_sql(name='covid_case_death', con=engine, if_exists='append', index=False)

In [24]:
#clean population data
pop_csv = "Resources/state_populations.csv"

In [35]:
pop_df = pd.read_csv(pop_csv)

pop_df.head()

,state,population
0,AL,"5,030,053"
1,AK,"736,081"
2,AZ,"7,158,923"
3,AR,"3,013,756"
4,CA,"39,576,757"


In [36]:
pop_df["population"] = pop_df["population"].str.replace(",","").astype(float)


In [37]:
pop_df

,state,population
0,AL,5030053.0
1,AK,736081.0
2,AZ,7158923.0
3,AR,3013756.0
4,CA,39576757.0
5,CO,5782171.0
6,CT,3608298.0
7,DE,990837.0
8,FL,21570527.0
9,GA,10725274.0


In [38]:
#add data to sql table
#pop_df.to_sql(name='state_population', con=engine, if_exists='append', index=False)